In [ ]:
!pip install fuzzywuzzy[speedup]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd


# Read the data from the CSV file
df = pd.read_csv("/content/Uroosha's names sheet - _A_ Only with _.csv")

# Split the "Investigators" column into separate rows by pipe delimiter
df["Unclubbed_Investigators"] = df["PI"].str.split("|")

# Explode the split "Unclubbed_Investigators" column
df = df.explode("Unclubbed_Investigators")

# Clean up the "Unclubbed_Investigators" column by removing leading/trailing spaces
df["Unclubbed_Investigators"] = df["Unclubbed_Investigators"].str.strip()

# # Further split each investigator entry by comma delimiter and create new columns
# df = pd.DataFrame(df["Unclubbed_Investigators"].str.split(",", expand=True))

# # Filter rows where "PRINCIPAL_INVESTIGATOR" is present in the first column
# filtered_df = df[df[df.columns[0]].str.contains("PRINCIPAL_INVESTIGATOR", na=False)]

# # Remove occurrences of prefixes and leading/trailing spaces in all columns
# filtered_df = filtered_df.apply(lambda row: row.str.strip().str.replace(r"^(A\/Prof\.|A\/Prof|Prof|Prof.|Dr\.)\s+", "", regex=True))

# # Keep only unique entries based on column 1
# filtered_df = filtered_df.drop_duplicates(subset=df.columns[1])

KeyError: 'PI'

In [ ]:
filtered_df=df["Unclubbed_Investigators"]

KeyError: 'Unclubbed_Investigators'

In [ ]:
# Save the modified DataFrame to a new CSV file with UTF-8 encoding
filtered_df.to_csv("name.csv", index=False, header=False, encoding='utf-8-sig')

# With Fuzzy String Match Implementation

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Read the data from the CSV file
df = pd.read_csv("Breast_cancer.csv")

# Split the "Investigators" column into separate rows by pipe delimiter
df["Unclubbed_Investigators"] = df["Investigators"].str.split("|")

# Explode the split "Unclubbed_Investigators" column
df = df.explode("Unclubbed_Investigators")

# Clean up the "Unclubbed_Investigators" column by removing leading/trailing spaces
df["Unclubbed_Investigators"] = df["Unclubbed_Investigators"].str.strip()

# Further split each investigator entry by comma delimiter and create new columns
df = pd.DataFrame(df["Unclubbed_Investigators"].str.split(",", expand=True))

# Filter rows where "PRINCIPAL_INVESTIGATOR" is present in the first column
filtered_df = df[df[df.columns[0]].str.contains("PRINCIPAL_INVESTIGATOR", na=False)]

# Remove occurrences of prefixes and leading/trailing spaces in all columns
filtered_df = filtered_df.apply(lambda row: row.str.strip().str.replace(r"^(A\/Prof\.|A\/Prof|Prof|Prof.|Dr\.)\s+", "", regex=True))

# Keep only unique entries based on column 1
filtered_df = filtered_df.drop_duplicates(subset=df.columns[1])

# Define a function to calculate fuzzy string similarity
def similar(a, b):
    return fuzz.token_sort_ratio(a, b)

# Function to find and remove duplicates based on fuzzy matching
def remove_duplicates(df):
    duplicates = set()
    unique_names = df[df.columns[1]].unique()
    for i, name1 in enumerate(unique_names):
        if name1 in duplicates:
            continue
        for j, name2 in enumerate(unique_names[i+1:], i+1):
            if similar(name1, name2) > 80:  # Adjust the threshold as needed
                duplicates.add(name2)
    df = df[~df[df.columns[1]].isin(duplicates)]
    return df

# Remove duplicates based on fuzzy string matching
filtered_df = remove_duplicates(filtered_df)

# Sort unique investigators in alphabetical order by column 1
filtered_df = filtered_df.sort_values(by=df.columns[1], ascending=True)

# Save the modified DataFrame to a new CSV file with UTF-8 encoding
filtered_df.to_csv("Breast_cancer_cleaned_fuzzy.csv", index=False, header=False, encoding='utf-8-sig')

print("Data is cleaned")

## With Fuzzy String Match Implementation and QCing the discarded data as well

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Read the data from the CSV file
df = pd.read_csv("Lung_cancer.csv")

# Split the "Investigators" column into separate rows by pipe delimiter
df["Unclubbed_Investigators"] = df["Investigators"].str.split("|")

# Explode the split "Unclubbed_Investigators" column
df = df.explode("Unclubbed_Investigators")

# Clean up the "Unclubbed_Investigators" column by removing leading/trailing spaces
df["Unclubbed_Investigators"] = df["Unclubbed_Investigators"].str.strip()

# Further split each investigator entry by comma delimiter and create new columns
df = pd.DataFrame(df["Unclubbed_Investigators"].str.split(",", expand=True))

# Filter rows where "PRINCIPAL_INVESTIGATOR" is present in the first column
filtered_df = df[df[df.columns[0]].str.contains("PRINCIPAL_INVESTIGATOR", na=False)]

# Drop the first column as it contains only "PRINCIPAL_INVESTIGATOR"
filtered_df = filtered_df.drop(filtered_df.columns[0], axis=1)

# Remove occurrences of prefixes and leading/trailing spaces in all columns
filtered_df = filtered_df.apply(lambda row: row.str.strip().str.replace(r"^(A\/Prof\.|A\/Prof|Prof|Prof.|Dr|Dr\.)\s+", "", regex=True))

# # Sort unique investigators in alphabetical order
filtered_df = filtered_df.sort_values(by=df.columns[1], ascending=True)

# Get full name from the first column and affiliations from merged remaining columns
filtered_df["Affiliations"] = filtered_df.iloc[:, 1:].apply(lambda x: ', '.join(x.dropna()), axis=1)
filtered_df["full_name"] = filtered_df.iloc[:, 0]
# Keep only the desired columns and remove duplicates
filtered_df = filtered_df[["full_name", "Affiliations"]].drop_duplicates(subset="full_name")

# Sort by full name
filtered_df = filtered_df.sort_values(by="full_name", ascending=True)

def similar_with(name1, aff1, name2, aff2, name_weight=0.8, aff_weight=0.2):
    name_similarity = fuzz.token_sort_ratio(name1, name2)
    aff_similarity = fuzz.token_sort_ratio(aff1, aff2)
    weighted_similarity = (name_similarity * name_weight) + (aff_similarity * aff_weight)
    return weighted_similarity

# Function to find and remove duplicates based on fuzzy matching with weighted components
def remove_duplicates(df):
    duplicates = set()
    unique_rows = df.values.tolist()
    discarded_rows = []

    for i, row1 in enumerate(unique_rows):
        if row1[0] in duplicates:
            continue
        for j, row2 in enumerate(unique_rows[i + 1:], i + 1):
            similarity = similar_with(row1[0], row1[1], row2[0], row2[1])
            if similarity > 85:  # Adjust the threshold as needed
                duplicates.add(row2[0])
                discarded_rows.append((row2, row1, similarity))

    filtered_df = df[~df[df.columns[0]].isin(duplicates)]
    return filtered_df, discarded_rows

# Remove duplicates based on fuzzy string matching including affiliation
filtered_df, discarded_rows = remove_duplicates(filtered_df)

# Clean the "Discarded_Row" and "Matched_Part" columns in discarded_rows
discarded_rows = [
    [", ".join([item for item in row[0] if item]), ", ".join([item for item in row[1] if item]), row[2]]
    for row in discarded_rows
]

# Save the discarded rows to a separate CSV file with entire row information
discarded_df = pd.DataFrame({
    "Discarded_Row": [item[0] for item in discarded_rows],
    "Matched_Part": [item[1] for item in discarded_rows],
    "Similarity_Percentage": [item[2] for item in discarded_rows]
})
discarded_df.to_csv("lung_cancer_discarded_names_fuzzy.csv", index=False)

# # Save the modified DataFrame to a new CSV file with UTF-8 encoding
filtered_df.to_csv("Lung_cancer_cleaned_fuzzy.csv", index=False, header=False, encoding='utf-8-sig')

print("Data is cleaned")

Data is cleaned
